# DDD.jl

This is a tutorial and demo of `DDD.jl`.

We start by loading `DDD`.

In [ ]:
using DDD

To start off, we need to set up some parameters that control certain aspects of the systems we will be simulating. These will be used to generate and control all aspects of the system we want to model.

There are various ways of creating these, but here we will create them using named constructors because we can leverage the fact that they perform validations and automatically calculate derived values.

In [ ]:
dlnParams = DislocationParameters(;
    mobility = mobBCC(),
    dragCoeffs = (edge = 1, screw = 1e-1, climb = 1e9),
    coreRad = 0.015 * sqrt(2),
    minSegLen = 0.15 * sqrt(2),
    maxSegLen = 1.5 * sqrt(2),
    coreEnergy = 1 / (4 * π) * log(0.015 * sqrt(2) / 3.5e-5),
    coreRadMag = 3.5e-4,
)

matParams = MaterialParameters(;
    crystalStruct = BCC(),
    μ = 1,
    μMag = 80e3,
    ν = 0.25,
)

femParams = FEMParameters(;
    type = DispatchRegularCuboidMesh(),
    order = LinearElement(),
    model = CantileverLoad(),
    dx = 43.0,
    dy = 37.0,
    dz = 31.0,
    mx = 17,
    my = 13,
    mz = 11
)

slipSystems = SlipSystem(;
    crystalStruct = BCC(),
    slipPlane = Float64[-1 1 ;1 -1;0 0],
    bVec = Float64[1 1;1 1;1 -1]
)

intParams = IntegrationParameters(;
    method = AdaptiveEulerTrapezoid(),
    abstol = dlnParams.collisionDist / 2,
    reltol = dlnParams.collisionDist / 2,
)

intTime = IntegrationTime(;
    dt = 0.0,
    time = 0.0
)

We can see the fields of these structures by using the `fieldnames()` function on their types.

In [ ]:
# Either on the type of the variable.
fieldnames(typeof(matParams))
# Or directly on the type itself.
fieldnames(DislocationParameters)

We can also browse the documentation for additional information. Say we want to find out more about the function `DislocationParameters`.

In [ ]:
?DislocationParameters

`?` is used to browse for the documentation of types and functions. When used on variables, it provides type information.

In [ ]:
?femParams

Note how the types of all fields are reported back. The summary also provides the type heirarchy of the variable. This information allows the compiler to optimise code and enables one of the most powerful features of Julia, multiple dispatch.

The fields in a structure can be accessed via the dot syntax in the same way it's done in Python and Matlab. 

Julia draws heavy inspiration from functiional languages, so a lot of its syntax is syntactic sugar for functions. For example, accessing fields via dot syntax is syntactic sugar for `getproperty()`. Other examples of syntactic sugar include accessing and setting the values of array entries. 

In [ ]:
femParams.model

`CantileverLoad()` is a concrete type. It is used to set the appropriate boundary conditions by using multiple dispatch later on.

With all the parameters in place, we can start generating the data to be used in the simulation. The variable `regularCuboidMesh` contains all the information regarding the mesh. The constructor uses the canonical node numbering to minimise the need for custom code.

In [ ]:
regularCuboidMesh = buildMesh(matParams, femParams)

We can easily visualise the surface nodes. Julia has a robust plotting library with multiple backends, we will use `plotlyjs()` because it generates interactive plots and has great 3D capabilities.

In [ ]:
using Plots
plotlyjs()
figFE = plotFEDomain(regularCuboidMesh)

With the `regularCuboidMesh` and `femParams`, we can define the boundary conditions and prepare the necessary structures.

In [ ]:
cantileverBC, forceDisplacement = Boundaries(femParams, regularCuboidMesh)

`Boundaries()` function accepts keyword arguments that let the user modify the boundary nodes, but we've used the defaults. Which can also be plotted.

In [ ]:
figBound = plotBoundaries(cantileverBC, regularCuboidMesh)

Now that we have our FE domain we can populate it with dislocations. First we create our initial loops, then we use those loops to create a network that follows the specifications set by loop structures.

In [ ]:
dx, dy, dz = regularCuboidMesh.dx, regularCuboidMesh.dy, regularCuboidMesh.dz
segLen = (dlnParams.minSegLen + dlnParams.maxSegLen) / 2

prismOct = DislocationLoop(;
    loopType = loopPrism(),
    numSides = 8,
    nodeSide = 1,
    numLoops = 20,
    segLen = segLen * ones(8),
    slipSystemIdx = 1,
    slipSystem = slipSystems,
    label = nodeTypeDln.(ones(Int, 8)),
    buffer = 0,
    range = [0 dx; 0 dy; 0 dz],
    dist = Rand()
)

shearPent = DislocationLoop(;
    loopType = loopShear(),
    numSides = 5,
    nodeSide = 2,
    numLoops = 30,
    segLen = segLen * ones(10),
    slipSystemIdx = 2,
    slipSystem = slipSystems,
    label = nodeTypeDln.(ones(Int, 10)),
    buffer = 0,
    range = [0 dx; 0 dy; 0 dz],
    dist = Rand()
)

network = DislocationNetwork((prismOct, shearPent))

We can visualise the loops on their own.

In [ ]:
dlnFig = plotNodes(
    prismOct,
    m = 2,
    l = 3,
    linecolor = :blue,
    markershape = :circle,
    markercolor = :blue,
    legend = false,
)
plotNodes!(
    dlnFig,
    shearPent,
    m = 2,
    l = 3,
    linecolor = :red,
    markerstyle = :square,
    markercolor = :red,
    legend = false,
)

We can visualise the network on its own, without the FE domain.

In [ ]:
networkFig = plotNodes(
    network,
    m = 2,
    l = 3,
    linecolor = :blue,
    markershape = :circle,
    markercolor = :blue,
    legend = false,
)

Or we can visualise the network with the FE domain around it.

In [ ]:
networkFEFig = plotNodes(
    regularCuboidMesh,
    network,
    m = 2,
    l = 3,
    linecolor = :blue,
    markershape = :circle,
    markercolor = :blue,
    legend = false,
)